In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DIR = './'

In [3]:
train_df = pd.read_csv(os.path.join('https://media.githubusercontent.com/media/ssilvapinochet/RecSysData/refs/heads/main/Archivos%20finales/train_w_title.csv'))
train_df.head()

,user_id,book_id,review_text,rating,book_title
0,4289,207,More chick lit than historical fiction; but I ...,1.00,The Other Boleyn Girl (The Plantagenet and Tud...
1,2344,833,This book was a lovely surprise. I don't know ...,0.75,The Golem and the Jinni (The Golem and the Jin...
2,5970,791,I was not liking this book from almost the fir...,0.25,"You (You, #1)"
3,911,216,I first read this for a college course called ...,0.50,Bridge to Terabithia
4,1952,447,I want book two like now.,1.00,"Snow Like Ashes (Snow Like Ashes, #1)"


In [4]:
test_df = pd.read_csv(os.path.join('https://media.githubusercontent.com/media/ssilvapinochet/RecSysData/refs/heads/main/Archivos%20finales/test_w_title.csv'))
test_df.head()

,user_id,book_id,review_text,rating,book_title
0,4054,236,Extremely well written in translation. Nuanced...,0.75,The Shadow of the Wind (The Cemetery of Forgot...
1,719,878,"I admire Jenny Lawson so much, even more now t...",0.75,Furiously Happy: A Funny Book About Horrible T...
2,3768,396,This book for me is a 4.5 stars books. I bough...,0.75,"Soulless (Parasol Protectorate, #1)"
3,4151,771,The rest of the trilogy could turn this into 5...,0.75,"Annihilation (Southern Reach, #1)"
4,6477,478,I do not understand what all the hype was abou...,0.25,The Underground Railroad


In [5]:
train_df.shape

(82430, 5)

In [6]:
reviews_df = train_df[['user_id', 'book_id', 'rating']]

# Crear una matriz de interacciones usuario-item
interaction_matrix = reviews_df.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

In [7]:
test_reviews_df = test_df[['user_id', 'book_id', 'rating']]
test_matrix = test_reviews_df.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

In [8]:
interaction_matrix.shape

(6786, 996)

In [9]:
interaction_matrix.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,987,988,989,990,991,992,993,996,997,999
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.00,0.75,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.75,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [11]:
#implementamos el modelo adversarial generativo
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [12]:
num_epochs = 200
real_label = 1
fake_label = 0

batch_size = 100
num_batches = len(interaction_matrix) // batch_size
lambda_r1 = 10
learning_rate = 0.0002
discriminator_updates = 1

user_dim = interaction_matrix.shape[1]
noise_dim = 100
generator = Generator(noise_dim, user_dim)
discriminator = Discriminator(user_dim)

g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
loss_function = nn.BCELoss()

for epoch in range(num_epochs):
    for i in range(num_batches):
        batch_data = interaction_matrix.iloc[i * batch_size:(i + 1) * batch_size]
        real_data = torch.tensor(batch_data.values, dtype=torch.float32, requires_grad=True)

        for _ in range(discriminator_updates):
            d_optimizer.zero_grad()

            real_output = discriminator(real_data)
            real_loss = loss_function(real_output, torch.full_like(real_output, real_label))

            # Ajuste por R1
            real_grad = torch.autograd.grad(
                outputs=real_output.sum(), inputs=real_data,
                create_graph=True, retain_graph=True, only_inputs=True
            )[0]
            r1_penalty = lambda_r1 * (real_grad.norm(2, dim=1) ** 2).mean()
            total_real_loss = real_loss + r1_penalty

            noise = torch.randn(batch_size, noise_dim)
            fake_data = generator(noise)
            fake_output = discriminator(fake_data.detach())
            fake_loss = loss_function(fake_output, torch.full_like(fake_output, fake_label))

            d_loss = total_real_loss + fake_loss
            d_loss.backward()
            d_optimizer.step()

        g_optimizer.zero_grad()
        fake_output = discriminator(fake_data)
        g_loss = loss_function(fake_output, torch.full_like(fake_output, real_label))
        g_loss.backward()
        g_optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}")

Epoch [1/200], d_loss: 0.8020725846290588, g_loss: 2.3236799240112305
Epoch [2/200], d_loss: 1.404470443725586, g_loss: 0.58762127161026
Epoch [3/200], d_loss: 1.368158221244812, g_loss: 0.6336700916290283
Epoch [4/200], d_loss: 1.3485352993011475, g_loss: 0.6649709939956665
Epoch [5/200], d_loss: 1.3330113887786865, g_loss: 0.6950263381004333
Epoch [6/200], d_loss: 1.3210794925689697, g_loss: 0.7219855785369873
Epoch [7/200], d_loss: 1.3083679676055908, g_loss: 0.74076247215271
Epoch [8/200], d_loss: 1.2718186378479004, g_loss: 0.7969896793365479
Epoch [9/200], d_loss: 1.2724308967590332, g_loss: 0.8148479461669922
Epoch [10/200], d_loss: 1.307234287261963, g_loss: 0.7436781525611877
Epoch [11/200], d_loss: 1.2978029251098633, g_loss: 0.7807615399360657
Epoch [12/200], d_loss: 1.2657935619354248, g_loss: 0.8440330028533936
Epoch [13/200], d_loss: 1.3058613538742065, g_loss: 0.7296472787857056
Epoch [14/200], d_loss: 1.3322474956512451, g_loss: 0.6938542127609253
Epoch [15/200], d_loss

In [13]:
def get_top_k_recommendations(user_id, generated_interactions, k=10):

  top_k_indices = np.argsort(generated_interactions)[::-1][:k]

  top_k_book_ids = interaction_matrix.columns[top_k_indices]

  return top_k_book_ids.tolist()

def generate_recommendations(generator, noise_dim, num_recommendations=10):
    recommendations = {}
    for user_id in interaction_matrix.index:
        noise = torch.randn(noise_dim)
        generated_interactions = generator(noise).detach().numpy()

        top_k_recommendations_for_user = get_top_k_recommendations(user_id, generated_interactions, k=num_recommendations)

        recommendations[user_id] = top_k_recommendations_for_user
    return recommendations

In [14]:
recommendations = generate_recommendations(generator, noise_dim)
user_id = 1
top_10_recommendations = recommendations.get(user_id)
print("Top 10 recommendations for user with ID 1")
print(top_10_recommendations)

Top 10 recommendations for user with ID 1
[1, 849, 118, 818, 416, 303, 305, 102, 534, 530]


In [47]:
recommendations

{1: [1, 849, 118, 818, 416, 303, 305, 102, 534, 530],
 2: [1, 206, 37, 51, 83, 858, 118, 120, 818, 155],
 3: [1, 453, 120, 664, 246, 341, 37, 342, 877, 29],
 4: [1, 29, 849, 83, 246, 858, 534, 263, 664, 877],
 5: [246, 534, 83, 29, 118, 120, 770, 25, 206, 225],
 6: [1, 25, 664, 186, 159, 155, 147, 120, 118, 114],
 7: [1, 147, 206, 186, 768, 770, 774, 776, 159, 155],
 8: [263, 118, 120, 206, 225, 246, 650, 303, 341, 530],
 9: [1, 342, 279, 29, 416, 37, 263, 51, 303, 246],
 10: [1, 246, 225, 221, 220, 219, 711, 719, 206, 186],
 11: [1, 530, 219, 221, 664, 225, 37, 650, 639, 638],
 12: [279, 650, 303, 83, 25, 534, 774, 225, 29, 118],
 13: [650, 83, 120, 354, 225, 206, 341, 118, 263, 303],
 14: [279, 206, 342, 650, 303, 350, 354, 25, 263, 29],
 15: [83, 350, 25, 341, 29, 650, 303, 534, 530, 279],
 16: [1, 118, 114, 341, 102, 818, 246, 83, 858, 263],
 17: [1, 534, 280, 279, 263, 638, 639, 246, 650, 664],
 18: [1, 342, 29, 279, 451, 449, 263, 37, 350, 246],
 19: [770, 118, 25, 279, 650, 29, 

In [15]:
def precision_at_k(recommendations, test_matrix, k=10):
    precisions = []
    for user_id, recommended_books in recommendations.items():
        if user_id in test_matrix.index:
            relevant_items = test_matrix.loc[user_id]
            relevant_items = relevant_items[relevant_items > 0].index
            hits = len(set(recommended_books[:k]).intersection(relevant_items))
            precisions.append(hits / k)
    return np.mean(precisions)

def recall_at_k(recommendations, test_matrix, k=10):
    recalls = []
    for user_id, recommended_books in recommendations.items():
        if user_id in test_matrix.index:
            relevant_items = test_matrix.loc[user_id]
            relevant_items = relevant_items[relevant_items > 0].index
            if len(relevant_items) > 0:
                hits = len(set(recommended_books[:k]).intersection(relevant_items))
                recalls.append(hits / len(relevant_items))
    return np.mean(recalls)

def map_at_k(recommendations, test_matrix, k=10):
    average_precisions = []
    for user_id, recommended_books in recommendations.items():
        if user_id in test_matrix.index:
            relevant_items = test_matrix.loc[user_id]
            relevant_items = relevant_items[relevant_items > 0].index
            hits, precision_sum = 0, 0
            for i, book_id in enumerate(recommended_books[:k]):
                if book_id in relevant_items:
                    hits += 1
                    precision_sum += hits / (i + 1)
            if hits > 0:
                average_precisions.append(precision_sum / min(len(relevant_items), k))
            else:
                average_precisions.append(0)
    return np.mean(average_precisions)

def ndcg_at_k(recommendations, test_matrix, k=10):
    ndcgs = []
    for user_id, recommended_books in recommendations.items():
        if user_id in test_matrix.index:
            relevant_items = test_matrix.loc[user_id]
            relevant_items = relevant_items[relevant_items > 0].index
            dcg, idcg = 0, 0
            for i, book_id in enumerate(recommended_books[:k]):
                if book_id in relevant_items:
                    dcg += 1 / np.log2(i + 2)
            for i in range(min(len(relevant_items), k)):
                idcg += 1 / np.log2(i + 2)
            if idcg > 0:
                ndcgs.append(dcg / idcg)
            else:
                ndcgs.append(0)
    return np.mean(ndcgs)

In [16]:
k = 10
precision = precision_at_k(recommendations, test_matrix, k)
recall = recall_at_k(recommendations, test_matrix, k)
map_score = map_at_k(recommendations, test_matrix, k)
ndcg = ndcg_at_k(recommendations, test_matrix, k)

print(f"Precision@{k}: {precision:.4f}")
print(f"Recall@{k}: {recall:.4f}")
print(f"MAP@{k}: {map_score:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")

Precision@10: 0.0078
Recall@10: 0.0231
MAP@10: 0.0070
NDCG@10: 0.0152


In [17]:
import gzip
import json
import re
import sys

def load_data(file_name, head = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)

            if (head is not None) and (count > head):
                break
    return data

In [18]:
import requests

DIR = './'

file_names = pd.read_csv(os.path.join('https://media.githubusercontent.com/media/ssilvapinochet/RecSysData/refs/heads/main/dataset_names.csv'))
display(file_names)

file_name_type_mapping = dict(zip(file_names['name'].values, file_names['type'].values))
file_name_url_mapping = {}

for fname in file_name_type_mapping:
    ftype = file_name_type_mapping[fname]
    if ftype == "complete":
        url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/'+fname
        file_name_url_mapping[fname] = url
    elif ftype == "byGenre":
        url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/'+fname
        file_name_url_mapping[fname] = url

from tqdm import tqdm

def download_by_name(fname, local_filename):
        if fname in file_name_url_mapping:
            url = file_name_url_mapping[fname]
            response = requests.get(url, stream=True)
            total_size_in_bytes= int(response.headers.get('content-length', 0))
            with open(local_filename, 'wb') as f:
                with tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True) as pbar:
                    for data in response.iter_content(chunk_size=8192):
                        pbar.update(len(data))
                        f.write(data)
            print('Dataset', fname, 'has been downloaded!')
        else:
            print('Dataset', fname, 'can not be found!')

OUT_DIR = "./books_meta"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

    output_path = os.path.join(OUT_DIR, 'goodreads_book_genres_initial.json.gz')
    download_by_name('goodreads_book_genres_initial.json.gz', output_path)

,type,name
0,complete,goodreads_book_works.json.gz
1,complete,goodreads_book_authors.json.gz
2,complete,goodreads_book_series.json.gz
3,complete,goodreads_books.json.gz
4,complete,goodreads_book_genres_initial.json.gz
5,byGenre,goodreads_books_children.json.gz
6,byGenre,goodreads_books_comics_graphic.json.gz
7,byGenre,goodreads_books_fantasy_paranormal.json.gz
8,byGenre,goodreads_books_history_biography.json.gz
9,byGenre,goodreads_books_mystery_thriller_crime.json.gz


100%|██████████| 24.3M/24.3M [00:00<00:00, 93.3MiB/s]

Dataset goodreads_book_genres_initial.json.gz has been downloaded!


In [45]:
books_meta_gr = load_data('./books_meta/goodreads_book_genres_initial.json.gz', 10000000)
book_genres = pd.DataFrame(books_meta_gr)
book_genres.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [20]:
item_counts = train_df["book_id"].value_counts()
total_interacciones = len(train_df)
items_popularity = item_counts / total_interacciones
items_popularity = items_popularity.to_dict()

In [21]:
book_id_map = {book_id: i for i, book_id in enumerate(train_df['book_id'].unique())}

In [43]:
def calculate_diversity(recommendations, book_genres, k=10):
    book_genres['dominant_genre'] = book_genres['genres'].apply(lambda x: next(iter(x), 'unknown') if x else 'unknown')
    book_id_to_genre = book_genres.set_index('book_id')['dominant_genre'].to_dict()
    diversities = []

    for user_id, recommended_books in recommendations.items():

        genres = {book_id_to_genre.get(str(book_id), 'unknown') for book_id in recommended_books[:k]}
        diversities.append(len(genres) / k if k > 0 else 0)

    return np.mean(diversities) if diversities else 0

def calculate_novelty(recommendations, item_popularity):
  novelties = []
  for user_id, recommended_books in recommendations.items():
      user_novelty = 0
      for book_id in recommended_books:
          if book_id in item_popularity:
              user_novelty += (1 - item_popularity[book_id])
          else:
              user_novelty += 1
      novelties.append(user_novelty / len(recommended_books))
  return np.mean(novelties)

In [46]:
diversity = calculate_diversity(recommendations, book_genres)
novelty = calculate_novelty(recommendations, items_popularity)

print(f"Average Diversity: {diversity:.4f}")
print(f"Average Novelty: {novelty:.4f}")

Average Diversity: 0.3649
Average Novelty: 0.9978
